# Projecting Regression coeffcients back onto the brain

In [1]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
from sklearn.cross_decomposition import CCA
import joblib

from nilearn import datasets as ds
from nilearn.maskers import NiftiLabelsMasker, NiftiMasker
from nilearn.image import resample_img, index_img
import nilearn.datasets as ds

import pandas as pd
import numpy as np

The scikit-learn version is 0.23.2.


In [2]:
# Loading will fail with newer sklearn versions > 0.24
pop_cca = joblib.load('../org_CCA_k25_lonely_full')

/Users/chloesavignac/.conda/envs/old_python/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.cross_decomposition.cca_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cross_decomposition. Anything that cannot be imported from sklearn.cross_decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/chloesavignac/.conda/envs/old_python/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CCA from version 0.21.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [3]:
date = '25.05.13'
print(date)

# Load regression coefficients for males and females
tables_m = pd.read_csv(f'{date}/tables_m.csv')
tables_f = pd.read_csv(f'{date}/tables_f.csv')

# Store interaction terms 
# MALES
interactions_m = tables_m[['HC-DN_pattern_PLS_C1', 'HC-DN_pattern_PLS_C2', 'HC-DN_pattern_PLS_C3']].reset_index(drop=True)
# Fix index 
interactions_m.index=interactions_m.index+1
# Save
interactions_m.to_csv(f'{date}/interactions_m_age_not_regressed_2025.csv')
int_m = pd.read_csv(f'{date}/interactions_m_age_not_regressed_2025.csv',index_col=0)

# Store interaction terms 
# FEMALES
interactions_f = tables_f[['HC-DN_pattern_PLS_C1', 'HC-DN_pattern_PLS_C2', 'HC-DN_pattern_PLS_C3']].reset_index(drop=True)
# Fix index 
interactions_f.index=interactions_f.index+1
# Save
interactions_f.to_csv(f'{date}/interactions_f_age_not_regressed_2025.csv')
int_f = pd.read_csv(f'{date}/interactions_f_age_not_regressed_2025.csv',index_col=0)

25.05.13


In [4]:
def inverse_transform(self, X, Y=None):
    # From pls space to original space
    X_reconstructed = np.matmul(X, self.x_loadings_.T)

    # From pls space to original space
    Y_reconstructed = np.matmul(Y, self.y_loadings_.T)

    return X_reconstructed, Y_reconstructed

In [5]:
df = pd.read_csv('../CCA_exp_PREVENTAD.csv', index_col=0)

In [6]:
cca_cols = []
for i in range(1,51):
    cca_cols.append(f"{i}")

In [7]:
print(df.Sex_Male.value_counts())

Sex_Male
0    250
1    182
Name: count, dtype: int64


In [8]:
# Store id for males and females
id_f = np.where(df.Sex_Female==1)
id_m = np.where(df.Sex_Male==1)
id_all = df.index

In [9]:
# -----------------------------
# Inverse transform for MALES
# -----------------------------

# Split into HC and DN features for males
hc = df[cca_cols].iloc[id_m].iloc[:, :25]
dn = df[cca_cols].iloc[id_m].iloc[:, 25:]

# Compute element-wise interactions between HC and intermediate components
hc_interactions = []
for int_component in int_m.columns:
    interactions = []
    for col in hc.columns:
        interaction = hc[col] * int_m[int_component][int(col)]
        interactions.append(interaction)
    hc_interactions.append(interactions)

hc_int_m = np.array(hc_interactions)
print("HC shape:", hc_int_m.shape)

# Compute element-wise interactions between DN and intermediate components
dn_interactions = []
for int_component in int_m.columns:
    interactions = []
    for col in dn.columns:
        interaction = dn[col] * int_m[int_component][int(col)]
        interactions.append(interaction)
    dn_interactions.append(interactions)

dn_int_m = np.array(dn_interactions)
print("DN shape:", dn_int_m.shape)

# Inverse transform and save the outputs for each of the first 3 components
for i in range(3):
    hc_recon, dn_recon = inverse_transform(pop_cca, hc_int_m[i].T, dn_int_m[i].T)

    np.savetxt(f'{date}/brain_imaging/interactions_pls_c{i+1}_hc_males_age_not_regressed_2025.csv', hc_recon, delimiter=",")
    np.savetxt(f'{date}/brain_imaging/interactions_pls_c{i+1}_dn_males_age_not_regressed_2025.csv', dn_recon, delimiter=",")


HC shape: (3, 25, 182)
DN shape: (3, 25, 182)


In [10]:
# -------------------------------
# Inverse transform for FEMALES
# -------------------------------

# Split into HC and DN features for females
hc = df[cca_cols].iloc[id_f].iloc[:, :25]
dn = df[cca_cols].iloc[id_f].iloc[:, 25:]

# Compute element-wise interactions between HC and intermediate components
hc_interactions = []
for int_component in int_f.columns:
    interactions = []
    for col in hc.columns:
        interaction = hc[col] * int_f[int_component][int(col)]
        interactions.append(interaction)
    hc_interactions.append(interactions)

hc_int_f = np.array(hc_interactions)
print("HC shape:", hc_int_f.shape)

# Compute element-wise interactions between DN and intermediate components
dn_interactions = []
for int_component in int_f.columns:
    interactions = []
    for col in dn.columns:
        interaction = dn[col] * int_f[int_component][int(col)]
        interactions.append(interaction)
    dn_interactions.append(interactions)

dn_int_f = np.array(dn_interactions)
print("DN shape:", dn_int_f.shape)

# Inverse transform and save the outputs for each of the first 3 components
for i in range(3):
    hc_recon, dn_recon = inverse_transform(pop_cca, hc_int_f[i].T, dn_int_f[i].T)

    np.savetxt(f'{date}/brain_imaging/interactions_pls_c{i+1}_hc_females_age_not_regressed_2025.csv', hc_recon, delimiter=",")
    np.savetxt(f'{date}/brain_imaging/interactions_pls_c{i+1}_dn_females_age_not_regressed_2025.csv', dn_recon, delimiter=",")


HC shape: (3, 25, 250)
DN shape: (3, 25, 250)
